In [1]:
!pip install dash
!pip install Flask

In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

In [3]:
df = pd.read_csv('transaction_data.csv')

In [4]:
df.head(5)

,Transaction ID,TimeStamp,Amount,Customer ID,Payment Method
0,999261,1698617917,1759.69,5655,PayPal
1,785175,1690195733,1237.34,11738,PayPal
2,731665,1695778565,1644.61,11167,Google Pay
3,330716,1681465168,923.89,8895,Google Pay
4,550474,1686770039,1192.04,12054,Credit Card


In [5]:
df['TimeStamp'] = pd.to_datetime(df['TimeStamp'],unit = 's')

In [6]:
app = dash.Dash(__name__)

In [7]:
app.layout = html.Div([
    html.H1('Transaction Dashboard'),
    dcc.Graph(id='tran_graphs'),
    dcc.Dropdown(
        id='mode_dropdown',
        options=[{'label':method , 'value':method} for method in df['Payment Method'].unique()],
        value = None,
        placeholder = "Select a Payment Mode"
        
    ),
])
@app.callback(
    Output('tran_graphs','figure'),
    [Input('mode_dropdown','value')]
)
def update_graph(payment_mode):
    if payment_mode is None:
        filtered_df = df
    else:
        filtered_df = df[df['Payment Method'] == payment_mode]
    grouped_data = filtered_df.groupby(pd.Grouper(key='TimeStamp',freq='M')).size().reset_index(name='Transaction Volume')
    
    fig = px.line(grouped_data, x='TimeStamp', y='Transaction Volume',title='Transaction volume over time')
    return fig

In [8]:
if __name__ =='__main__':
    app.run_server(debug = True)